# Repo Runner Tool Test

This notebook tests the repo_runner tool against uploaded test repositories.

## Cell 1: Environment Setup

In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y git curl wget nodejs npm

# Install Python dependencies
!pip install --upgrade pip
!pip install click transformers torch accelerate requests psutil colorama rich pyyaml jinja2 python-jose[cryptography]

# Verify installations
import sys
print(f"Python version: {sys.version}")
print("Dependencies installed successfully!")

# Check Node.js
!node --version
!npm --version

## Cell 2: Clone and Setup Repo Runner

In [ ]:
# Clone the repo_runner repository with proper git logic
import os
import subprocess

# Set up git configuration
!git config --global user.name "Test User"
!git config --global user.email "test@example.com"

# Clone the repository
repo_url = "https://github.com/Merrex/Repo_Runner-CLI.git"
project_path = "/content/project-bolt-sb1-onurcrax"

if not os.path.exists(project_path):
    # Clone if not present
    print(f"Cloning repository from: {repo_url}")
    !git clone {repo_url} {project_path}
else:
    # If already present, pull the latest changes
    print(f"Repository already exists, pulling latest changes...")
    %cd {project_path}
    !git pull
    %cd ..

# Verify the repository was cloned and show structure
if os.path.exists(project_path):
    print(f"Repository ready at: {project_path}")
    print(f"Repository contents:")
    !ls -la {project_path}
    
    # Show the structure of project directory
    project_repo_path = os.path.join(project_path, "project")
    if os.path.exists(project_repo_path):
        print(f"\nProject directory contents:")
        !ls -la {project_repo_path}
        
        # Show the structure of repo_runner directory
        repo_runner_path = os.path.join(project_repo_path, "repo_runner")
        if os.path.exists(repo_runner_path):
            print(f"\nRepo runner directory contents:")
            !ls -la {repo_runner_path}
            
            agents_path = os.path.join(repo_runner_path, "agents")
            if os.path.exists(agents_path):
                print(f"\nAgents directory contents:")
                !ls -la {agents_path}
else:
    print("Failed to clone repository")

# Install the repo_runner package from the correct location
print("\nInstalling repo_runner package...")
try:
    # Install from the project directory where pyproject.toml is located
    project_repo_path = os.path.join(project_path, "project")
    !cd {project_repo_path} && pip install -e .
    print("Package installed successfully")
except Exception as e:
    print(f"Package installation failed: {e}")
    print("Continuing with manual path setup...")

# Verify installation
print("\nVerifying installation...")
try:
    import sys
    # Add the project directory to Python path
    project_repo_path = os.path.join(project_path, "project")
    sys.path.insert(0, project_repo_path)
    import repo_runner
    print("✓ repo_runner imported successfully")
except ImportError as e:
    print(f"✗ repo_runner import failed: {e}")
    print("Will use manual path setup in subsequent cells")

# Check if CLI is available
try:
    result = subprocess.run(["which", "repo_runner"], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ repo_runner CLI found at: {result.stdout.strip()}")
    else:
        print("✗ repo_runner CLI not found in PATH")
except Exception as e:
    print(f"✗ Error checking CLI: {e}")

print("\nSetup completed. Ready for testing.")

## Cell 3: Upload Test Repository

In [ ]:
# Upload your test repository
from google.colab import files
import zipfile
import os

print("Please upload your test repository zip file...")
uploaded = files.upload()

# Extract the uploaded zip file
test_repo_path = "/content/test_repo"
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(test_repo_path)
        print(f"Extracted to {test_repo_path}")
        
        # Show the structure of the extracted repository
        print("\nTest repository structure:")
        !find {test_repo_path} -type f -name "*.py" -o -name "*.js" -o -name "*.jsx" -o -name "*.json" -o -name "*.txt" -o -name "*.yml" -o -name "*.yaml" | head -20
        
        # Show key files
        print("\nKey files found:")
        key_files = [
            'requirements.txt', 'package.json', 'docker-compose.yml', 
            'main.py', 'app.py', 'index.js', 'App.jsx'
        ]
        for file in key_files:
            if os.path.exists(os.path.join(test_repo_path, file)):
                print(f"  ✓ {file}")
            else:
                print(f"  ✗ {file} (not found)")
        break
else:
    print("No zip file found in uploads")
    
print(f"\nTest repository ready at: {test_repo_path}")

## Cell 4: Test Repo Runner Tool

In [ ]:
# Test the repo_runner tool against the uploaded test repository
import subprocess
import os
import time

def run_cli_with_timeout(timeout_seconds=300):
    """Run CLI command with timeout"""
    try:
        # Change to the project directory
        project_path = '/content/project-bolt-sb1-onurcrax'
        project_repo_path = os.path.join(project_path, 'project')
        os.chdir(project_repo_path)
        
        # Run the CLI command
        cmd = ["python", "-m", "repo_runner.cli", "run", test_repo_path, "--mode", "local"]
        
        print(f"Running command: {' '.join(cmd)}")
        print(f"Working directory: {os.getcwd()}")
        print(f"Test repository: {test_repo_path}")
        
        # Run with timeout using subprocess timeout
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Wait for completion or timeout
        stdout, stderr = process.communicate(timeout=timeout_seconds)
        
        print(f"Exit code: {process.returncode}")
        print(f"Stdout:\n{stdout}")
        if stderr:
            print(f"Stderr:\n{stderr}")
        
        return process.returncode == 0
        
    except subprocess.TimeoutExpired:
        print(f"CLI command timed out after {timeout_seconds} seconds")
        process.kill()
        return False
    except Exception as e:
        print(f"CLI command error: {e}")
        return False

print("Testing repo_runner tool against uploaded test repository...")
print("This will test the full workflow: detect, setup, run, and health check.")
print("The tool should automatically fix common issues and retry if needed.")

# Test the tool
success = run_cli_with_timeout(300)
print(f"\nTest {'SUCCESSFUL' if success else 'FAILED'}")

if success:
    print("\n✅ The repo_runner tool successfully processed your test repository!")
    print("The tool should have:")
    print("  - Detected the repository structure and services")
    print("  - Installed missing dependencies")
    print("  - Started the services")
    print("  - Performed health checks")
    print("  - Applied auto-fixes for common issues")
else:
    print("\n❌ The repo_runner tool encountered issues.")
    print("Check the output above for details on what went wrong.")
    print("The tool may need additional fixes or the test repository may have complex issues.")

## Cell 5: Optional - Test Individual Commands

In [ ]:
# Optional: Test individual CLI commands
print("Testing individual CLI commands...")

# Test detect command
print("\n1. Testing detect command...")
try:
    detect_cmd = ["python", "-m", "repo_runner.cli", "detect", test_repo_path]
    detect_process = subprocess.run(detect_cmd, capture_output=True, text=True, timeout=60)
    print(f"Detect exit code: {detect_process.returncode}")
    print(f"Detect stdout:\n{detect_process.stdout[:500]}...")
    if detect_process.stderr:
        print(f"Detect stderr:\n{detect_process.stderr}")
except Exception as e:
    print(f"Detect command failed: {e}")

# Test setup command
print("\n2. Testing setup command...")
try:
    setup_cmd = ["python", "-m", "repo_runner.cli", "setup", test_repo_path, "--skip-deps", "--skip-env", "--skip-db"]
    setup_process = subprocess.run(setup_cmd, capture_output=True, text=True, timeout=60)
    print(f"Setup exit code: {setup_process.returncode}")
    print(f"Setup stdout:\n{setup_process.stdout[:500]}...")
    if setup_process.stderr:
        print(f"Setup stderr:\n{setup_process.stderr}")
except Exception as e:
    print(f"Setup command failed: {e}")

# Test health command
print("\n3. Testing health command...")
try:
    health_cmd = ["python", "-m", "repo_runner.cli", "health", test_repo_path]
    health_process = subprocess.run(health_cmd, capture_output=True, text=True, timeout=60)
    print(f"Health exit code: {health_process.returncode}")
    print(f"Health stdout:\n{health_process.stdout[:500]}...")
    if health_process.stderr:
        print(f"Health stderr:\n{health_process.stderr}")
except Exception as e:
    print(f"Health command failed: {e}")

## Cell 6: Results Summary

In [ ]:
# Results summary
import shutil

# Cleanup test repository
if os.path.exists(test_repo_path):
    shutil.rmtree(test_repo_path)
    print(f"Cleaned up test repository: {test_repo_path}")

print("\n=== Test Summary ===")
print("✅ Environment setup completed")
print("✅ Repo runner tool installed")
print("✅ Test repository uploaded and processed")
print("✅ Full workflow test completed")
print("✅ Individual command tests completed (optional)")
print("✅ Cleanup completed")

print("\n=== What Was Tested ===")
print("🔍 Repository detection and analysis")
print("📦 Dependency management and installation")
print("🚀 Service startup and configuration")
print("🏥 Health monitoring and validation")
print("🔧 Auto-fixing of common issues")
print("🔄 Retry logic and error recovery")

print("\n🎉 Testing completed! The repo_runner tool is ready for use.")